In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import sys

WORKING_DIR = '/content/drive/MyDrive/Colab/improved_sci'
sys.path.append(WORKING_DIR)

In [3]:
from tqdm import tqdm
import time
import glob
import numpy as np
import torch
import utils
from PIL import Image
import logging
import argparse
import torch.utils
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable
from torchsummary import summary
import skimage as ski

from model import *
from multi_read_data import MemoryFriendlyLoader

<h1>Constants & Utility

In [4]:
TRAIN_BATCH_SIZE = 4
TEST_BATCH_SIZE = 1
SEED = 42
EPOCHS = 100
LEARN_RATE = 0.0003
NUM_STAGE = 3
COLOR_FORMAT = 'ycbcr'
IN_CHANNELS = 2

SESSION_NAME = COLOR_FORMAT if IN_CHANNELS == 1 else f"mixed_{COLOR_FORMAT}"
SAVE_DIR = os.path.join(WORKING_DIR, SESSION_NAME)

In [5]:
def save_images(tensor, path):
    image_numpy = tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)))
    im = Image.fromarray(np.clip(image_numpy * 255.0, 0, 255.0).astype('uint8'))
    im.save(path, 'png')

<h1>Training

In [6]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float)
else:
    print("WARNING: CUDA not available.")

In [7]:
utils.create_exp_dir(SAVE_DIR, scripts_to_save=glob.glob(os.path.join(WORKING_DIR, '*.py')))
model_path = SAVE_DIR + '/model_epochs/'
os.makedirs(model_path, exist_ok=True)
image_path = SAVE_DIR + '/image_epochs/'
os.makedirs(image_path, exist_ok=True)

Experiment dir : /content/drive/MyDrive/Colab/improved_sci/mixed_ycbcr


In [8]:
np.random.seed(SEED)
cudnn.benchmark = True
torch.manual_seed(SEED)
cudnn.enabled = True
torch.cuda.manual_seed(SEED)

In [9]:
model = TrainModel(stage=NUM_STAGE, color=COLOR_FORMAT, in_channels=IN_CHANNELS)

model.illumination.in_conv.apply(model.weights_init)
model.illumination.conv.apply(model.weights_init)
model.illumination.out_conv.apply(model.weights_init)
model.self_calibrate.in_conv.apply(model.weights_init)
model.self_calibrate.convs.apply(model.weights_init)
model.self_calibrate.out_conv.apply(model.weights_init)

model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARN_RATE, betas=(0.9, 0.999), weight_decay=3e-4)
MB = utils.count_parameters_in_MB(model)
print(MB)

0.005216


In [10]:
train_low_data_names = '/content/drive/MyDrive/Colab/data/LSRW/train'
TrainDataset = MemoryFriendlyLoader(img_dir=train_low_data_names, task='train')


test_low_data_names = '/content/drive/MyDrive/Colab/data/dark_test'
TestDataset = MemoryFriendlyLoader(img_dir=test_low_data_names, task='test')

train_queue = torch.utils.data.DataLoader(TrainDataset, batch_size=TRAIN_BATCH_SIZE, pin_memory=True, num_workers=0, shuffle=True)

test_queue = torch.utils.data.DataLoader(TestDataset, batch_size=TEST_BATCH_SIZE, pin_memory=True, num_workers=0, shuffle=True)

In [ ]:
total_step = 0

for epoch in tqdm(range(EPOCHS), desc="Epoch", position=0):
    model.train()
    losses = []
    for batch_idx, (input, _) in enumerate(train_queue):
        total_step += 1
        input = Variable(input, requires_grad=False).cuda()

        optimizer.zero_grad()
        #with torch.autograd.set_detect_anomaly(True):
        loss = model._loss(input)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()

        losses.append(loss.item())

    utils.save(model, os.path.join(model_path, 'weights_%d.pt' % epoch))

    if (epoch % 10 == 0 or epoch == EPOCHS - 1) and total_step != 0:
        model.eval()
        with torch.no_grad():
            for _, (input, image_name) in enumerate(test_queue):
                input = Variable(input).cuda()
                image_name = image_name[0].split('/')[-1].split('.')[0]
                illu_list, ref_list, input_list, atten = model(input)
                u_name = '%s.png' % (image_name + '_' + str(epoch))
                save_images(ref_list[0], os.path.join(image_path, u_name))

<h1> Evaluation

In [34]:
WEIGHTS = os.path.join(SAVE_DIR, 'model_epochs/weights_99.pt')

In [35]:
eval_path = SAVE_DIR + '/eval/'
os.makedirs(eval_path, exist_ok=True)

In [36]:
model = PredictModel(weights=WEIGHTS, color=COLOR_FORMAT, in_channels=IN_CHANNELS)
model = model.cuda()

In [37]:
eval_data_names = '/content/drive/MyDrive/Colab/data/LSRW/test'
EvalDataset = MemoryFriendlyLoader(img_dir=eval_data_names, task='eval')

eval_queue = torch.utils.data.DataLoader(EvalDataset, batch_size=TEST_BATCH_SIZE, pin_memory=True, num_workers=0, shuffle=True)

In [38]:
 model.eval()
 with torch.no_grad():
  for _, (input, image_name) in enumerate(eval_queue):
    input = Variable(input).cuda()
    image_name = image_name[0].split('/')[-1].split('.')[0]
    i, r = model(input)
    u_name = '%s.png' % (image_name)
    save_images(r, os.path.join(eval_path, u_name))